In [1]:
import re
import time
import random
import requests
import urllib.request
import pandas as pd
import json
import numpy as np
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

In [ ]:
df_thavesh = pd.read_excel(
    "/home/ammar/INVOKE/calon_PRN_scrapper/calon_prn_15_thavesh.xlsx"
)
df_thavesh

In [2]:
with open("duns_json.json", "r") as file:
    duns_db = json.load(file)
df_duns = pd.DataFrame(duns_db["rows"])

with open("parties_json.json", "r") as file:
    party_db = json.load(file)
df_party = pd.DataFrame(party_db["rows"])

In [4]:
df_party

,id,name,name_1,abbreviation
0,2,BARISAN NASIONAL,Barisan Nasional,BN
1,20,PARTI BERSATU RAKYAT SABAH,Barisan Nasional,BN
2,12,PARTI BUMIPUTERA PERKASA MALAYSIA,Gerakan Tanah Air,GTA
3,4,PARTI PEJUANG TANAHAIR,Gerakan Tanah Air,GTA
4,14,PARTI KESEJAHTERAAN DEMOKRATIK MASYARAKAT,Lain-lain,LAIN
5,23,PARTI SEDAR RAKYAT SARAWAK,Lain-lain,LAIN
6,22,PARTY BUMI KENYALANG,Lain-lain,LAIN
7,21,PARTI PERPADUAN RAKYAT SABAH,Lain-lain,LAIN
8,19,PARTI SOSIALIS MALAYSIA,Lain-lain,LAIN
9,18,PARTI UTAMA RAKYAT,Lain-lain,LAIN


In [131]:
# THIS CODE IS TO ADD NEW INFORMATION INTO THE TABLE
# DON'T RUN IF YOU DON'T HAVE ONE
new_info = {
    "dunId": 614,
    "parliamentId": 25,
    "dunCode": "N21",
    "dunName": "Pantai Irama",
    "parliamentCode": "P025",
    "stateId": 3,
    "stateName": "Kelantan",
    "parliamentName": "Bachok",
}

df_duns.loc[len(df_duns)] = pd.Series(new_info)

In [134]:
df_duns[df_duns["dunId"] == 614]

,dunId,dunCode,dunName,parliamentId,parliamentCode,parliamentName,stateId,stateName,dunCodeName
613,614,N21,Pantai Irama,25,P025,Bachok,3,Kelantan,N21 Pantai Irama


In [3]:
df_duns["dunCodeName"] = df_duns["dunCode"] + " " + df_duns["dunName"]

In [108]:
df_party_new = df_party[
    df_party["name"].isin(
        [
            "BARISAN NASIONAL",
            "PERIKATAN NASIONAL",
            "PAKATAN HARAPAN",
            "BEBAS",
            "PARTI RAKYAT MALAYSIA",
            "PARTI SOSIALIS MALAYSIA",
        ]
    )
]
# df_party_new["upper_party"] = df_party["name_1"].apply(str.upper)
# df_added_party = pd.DataFrame({"id":[24], "name":["LAIN-LAIN"], "name_1":["LAIN-LAIN"], "abbreviation":["LAIN-LAIN"]})

# ADD LAIHN LAIN INTO PARTY DF
df_party_new.loc[len(df_party_new)] = pd.Series(
    {"id": "24", "name": "LAIN-LAIN", "name_1": "Lain-lain", "abbreviation": "LAIN"}
)
df_party_new
# 24

/tmp/ipykernel_1302/2517395838.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_party_new.loc[len(df_party_new)] = pd.Series({"id":"24", "name":"LAIN-LAIN", "name_1":"Lain-lain", "abbreviation":"LAIN"})


,id,name,name_1,abbreviation
0,2,BARISAN NASIONAL,Barisan Nasional,BN
8,19,PARTI SOSIALIS MALAYSIA,Lain-lain,LAIN
15,9,PARTI RAKYAT MALAYSIA,Lain-lain,LAIN
18,5,BEBAS,Lain-lain,LAIN
20,1,PAKATAN HARAPAN,Pakatan Harapan,PH
22,3,PERIKATAN NASIONAL,Perikatan Nasional,PN
6,24,LAIN-LAIN,Lain-lain,LAIN


In [161]:
# TODO:
"""
1. N21 Pantai Irama -- not existed in DB

"""
# DUN MAPPING
df_thavesh["dun"] = df_thavesh["dun"].replace(
    "N06 Kuala Kubu Baharu", "N06 Kuala Kubu Bharu"
)  # rename
df_thavesh_dun = df_thavesh.merge(
    df_duns, left_on="dun", right_on="dunCodeName", how="left"
)
df_thavesh_dun = df_thavesh_dun[
    [
        "state",
        "dun",
        "name",
        "party",
        "dunId",
        "dunName",
    ]
]
# COALITION REPLACEMENT
df_thavesh_dun["party"] = df_thavesh_dun["party"].replace(
    "BARISAN NASIONAL OF MALAYSIA", "BARISAN NASIONAL"
)
df_thavesh_dun["party"] = df_thavesh_dun["party"].replace(
    "PARTI ISLAM SE MALAYSIA", "PERIKATAN NASIONAL"
)
df_thavesh_dun["party"] = df_thavesh_dun["party"].replace(
    "IKATAN DEMOKRATIK MALAYSIA", "PAKATAN HARAPAN"
)
df_thavesh_dun["party"] = df_thavesh_dun["party"].replace(
    "PARTI SOSIALIS MALAYSIA (BARU)", "PARTI SOSIALIS MALAYSIA"
)
df_thavesh_dun["party"] = df_thavesh_dun["party"].replace(
    "PENANG FRONT PARTY", "LAIN-LAIN"
)
df_thavesh_dun["party"] = df_thavesh_dun["party"].replace(
    "PARTI UTAMA RAKYAT", "LAIN-LAIN"
)


# PARTY MAPPING
df_thavesh_dun_party = df_thavesh_dun.merge(
    df_party_new, left_on="party", right_on="name", how="left"
)
df_thavesh_dun_party["party_id"] = df_thavesh_dun_party["id"]
df_thavesh_dun_party["dun_id"] = df_thavesh_dun_party["dunId"]
df_thavesh_dun_party["name"] = df_thavesh_dun_party["name_x"]
df_thavesh_dun_party["parliament_id"] = np.NaN

df_thavesh_dun_party = df_thavesh_dun_party[
    ["name", "party_id", "dun_id", "parliament_id"]
]
df_thavesh_dun_party
# df_thavesh_dun_party.isna().sum()
# df_thavesh_dun_party[df_thavesh_dun_party["dun_id"].isna()]

,name,party_id,dun_id,parliament_id
0,SHAMSILAH BINTI SIRU,3,16,NaN
1,SAFWAN HANIF BIN SHAFIE,5,16,NaN
2,HISHAM SUHAILY BINTI OTHMAN,2,16,NaN
3,ZULFADLI BIN MOHD YUSOFF,5,16,NaN
4,AMAR PARED BIN MAHAMUD,3,17,NaN
...,...,...,...,...
565,ABD RAZAK BIN AB SAID,2,371,NaN
566,TENGKU ABDULLAH BIN TENGKU RAKMAN,3,372,NaN
567,SUHAIMIZAN BIN BIZAR,2,372,NaN
568,YONG LI YI,3,373,NaN


In [160]:
import simplejson as sjson

df_dict = df_thavesh_dun_party.to_dict(orient="records")
with open("final_calon_prn_list.json", "w") as file:
    sjson.dump(df_dict, file, ignore_nan=True)

In [143]:
df_thavesh_dun_party

,name,party_id,dun_id,parliament_id
0,SHAMSILAH BINTI SIRU,3,16,null
1,SAFWAN HANIF BIN SHAFIE,5,16,null
2,HISHAM SUHAILY BINTI OTHMAN,2,16,null
3,ZULFADLI BIN MOHD YUSOFF,5,16,null
4,AMAR PARED BIN MAHAMUD,3,17,null
...,...,...,...,...
565,ABD RAZAK BIN AB SAID,2,371,null
566,TENGKU ABDULLAH BIN TENGKU RAKMAN,3,372,null
567,SUHAIMIZAN BIN BIZAR,2,372,null
568,YONG LI YI,3,373,null
